In [1]:
!pip install feedparser

     |████████████████████████████████| 80 kB 141 kB/s eta 0:00:01
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6064 sha256=e55a5f9dac18091bff0437d7854d2de0c28ee9545419b9a7d034317e1a675a76
  Stored in directory: /home/benson/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built sgmllib3k


In [2]:
!pip install newspaper3k

     |████████████████████████████████| 211 kB 93 kB/s eta 0:00:01
     |████████████████████████████████| 7.4 MB 210 kB/s eta 0:00:01
     |████████████████████████████████| 48 kB 184 kB/s eta 0:00:011
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398406 sha256=50252d4b1941daee623874f553673b754551554d0fdc2e980c17558ba3d99be6
  Stored in directory: /home/benson/.cache/pip/wheels/4c/91/46/3c208287b726df325a5979574324878b679116e4baae1af3c3
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=178bf0f73a34e9a6194d0ab09a322fceba0bd338b02e5355ba69ba16f286b9a2
  Stored in directory: /home/benson/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3355 sha256=90f858945fbf65a01d292a164a9a2b33f0bae655e9045752a5dad07a37869f48
  Stored in directory: /home/benson/.cache/pip/wheels/7f/d4/8f/6e2ca54744c9d7292d88ddb8d42876

In [1]:
import json

In [2]:
dictionary = {
  "cnn": {
    "link": "http://edition.cnn.com/"
  },
  "bbc": {
    "rss": "http://feeds.bbci.co.uk/news/rss.xml",
    "link": "http://www.bbc.com/"
  },
  "theguardian": {
    "rss": "https://www.theguardian.com/uk/rss",
    "link": "https://www.theguardian.com/international"
  },
  "breitbart": {
    "link": "http://www.breitbart.com/"
  },
  "infowars": {
    "link": "https://www.infowars.com/"
  },
  "foxnews": {
    "link": "http://www.foxnews.com/"
  },
  "nbcnews": {
    "link": "http://www.nbcnews.com/"
  },
  "washingtonpost": {
    "rss": "http://feeds.washingtonpost.com/rss/world",
    "link": "https://www.washingtonpost.com/"
  },
  "theonion": {
    "link": "http://www.theonion.com/"
  }
}

In [3]:
json_object = json.dumps(dictionary, indent = 4) 

In [4]:
with open("NewsPapers.json", "w") as outfile: 
    outfile.write(json_object) 

In [5]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime

# Set the limit for number of articles to download
LIMIT = 10000

data = {}
data['newspapers'] = {}

# Loads the JSON files with news sites
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

count = 1

# Iterate through each news company
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data.
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['text'] = content.text
                newsPaper['articles'].append(article)
                print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.
            if content.publish_date is None:
                print(count, " Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 100:
                    print("Too many noneType dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            newsPaper['articles'].append(article)
            print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper

# Finally it saves the articles as a JSON-file.
try:
    with open('scraped_articles.json', 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)

Building site for  cnn
1  Article has date of type None...
2  Article has date of type None...
3 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/26/entertainment/baby-yoda-gratitude-pop-culture-plc/index.html
4 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/19/entertainment/holiday-movies-christmas-specials-to-watch/index.html
5 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/27/entertainment/courteney-cox-friends-thanksgiving/index.html
6 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/27/entertainment/the-mandalorian-the-jedi-episode/index.html
7 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/27/entertainment/the-masked-singer-reveal-trnd/index.html
8 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/26/entertainment/friends-thanksgiving-episodes/index.html
9 articles dow

60 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/25/football/pele-tribute-diego-maradona-spt-intl/index.html
61 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/25/football/diego-maradona-death-argentina-spt-intl/index.html
62 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/25/football/gallery/diego-maradona/index.html
63 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/27/sport/manchester-united-cyber-attack-spt-intl/index.html
64 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/27/sport/markus-paul-tributes-dallas-cowboys-nfl-spt-intl/index.html
65 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/28/football/barcelona-real-madrid-el-clasico-2010-anniversary-cmd-spt-intl/index.html
66 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/11/27/spor

115 articles downloaded from cnn  using newspaper, url:  https://cnn.com/2020/11/25/football/diego-maradona-death-argentina-spt-intl/index.html
116 articles downloaded from cnn  using newspaper, url:  https://cnn.com/2020/11/24/football/bruno-fernandes-champions-league-manchester-united-spt-intl/index.html
117 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2020/10/23/football/pele-80-soccer-brazil-birthday-spt-intl/index.html
118 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2020/11/25/football/diego-maradona-death-argentina-spt-intl/index.html
119 articles downloaded from cnn  using newspaper, url:  https://www.cnn.com/2020/11/25/us/markus-paul-dallas-cowboys-trnd/index.html
120 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2017/03/04/vr/how-to-watch-vr
121 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2019/07/17/vr/apollo-moon-vr/index.html
122 articles downloaded from cnn  

Article `download()` failed with 404 Client Error: Not Found for url: https://money.cnn.com/2020/11/24/economy/brexit-no-deal-covid-pandemic/index.html on URL https://money.cnn.com/2020/11/24/economy/brexit-no-deal-covid-pandemic/index.html
continuing...
Article `download()` failed with 404 Client Error: Not Found for url: https://money.cnn.com/2020/11/17/economy/global-economy-coronavirus-bailout-imf-annual-report/index.html on URL https://money.cnn.com/2020/11/17/economy/global-economy-coronavirus-bailout-imf-annual-report/index.html
continuing...
Article `download()` failed with 404 Client Error: Not Found for url: https://money.cnn.com/2020/11/16/economy/asian-economies-japan-china-intl-hnk/index.html on URL https://money.cnn.com/2020/11/16/economy/asian-economies-japan-china-intl-hnk/index.html
continuing...
142 articles downloaded from cnn  using newspaper, url:  https://money.cnn.com/videos/business/2020/11/09/virgin-hyperloop-first-human-test-orig.cnn-business
Article `download

193 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/05/11/us/2020-atlantic-hurricane-season-fast-facts/index.html
194 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/08/14/us/gallery/western-wildfires-2020/index.html
195 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/10/08/world/antarctic-ozone-hole-scn-trnd/index.html
196 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2020/09/14/weather/hottest-summer-on-record-northern-hemisphere-trnd-scn/index.html
197 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/videos/world/2020/11/27/iran-nuclear-scientist-killed-tehran-vpx.cnn/video/playlists/around-the-world/
198 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/videos/world/2020/11/27/astrazeneca-north-korea-coronavirus-vaccine-suspected-cyberattack-sot-vpx-nr.cnn/video/playlists/around-the-world/
Article `down

...

38  Article has date of type None...
39  Article has date of type None...
40  Article has date of type None...
41  Article has date of type None...
42  Article has date of type None...
43 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/politics/national-security/when-he-leaves-office-can-ex-president-trump-be-trusted-n1248994
44 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/news/latino/young-latinos-mobilized-voted-were-pivotal-2020-organizers-want-keep-n1246853
45 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/slideshow/94th-macy-s-thanksgiving-parade-goes-ahead-without-crowds-n1249105
46 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/politics/2020-election/biden-predicted-gop-epiphany-after-election-trump-s-standing-way-n1248996
47 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/news/us-news/scott-peterson-thousands-c

96 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/veteran-services/next-steps-for-vets/12-steps-starting-small-business-n1001316
97 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/veteran-services/next-steps-for-vets/military-families-say-their-top-concern-n1010916
98 articles downloaded from nbcnews  using newspaper, url:  https://www.cnbc.com/2019/01/28/cnbc-and-acorns-announce-strategic-partnership.html
99  Article has date of type None...
100  Article has date of type None...
101 articles downloaded from nbcnews  using newspaper, url:  https://www.today.com/series/veterans/veterans-day-why-vets-struggle-talk-their-kids-t198476
102 articles downloaded from nbcnews  using newspaper, url:  https://www.today.com/series/veterans/history-veterans-day-how-safely-celebrate-kids-t198214
103  Article has date of type None...
104  Article has date of type None...
105  Article has date of type None...
106 articles downloaded fr

154 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/better/lifestyle/under-20-11-gifts-thoughtful-givers-budget-ncna825556
155 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/better/lifestyle/18-travel-gifts-make-traveling-better-easier-2019-ncna1087261
156 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/better/pop-culture/inexpensive-gifts-your-coworkers-will-love-ncna947591
157 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/better/lifestyle/15-gadgets-gifts-will-take-your-fitness-routine-next-level-ncna825381
158 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/better/business/how-get-out-debt-build-wealth-snowball-ncna864111
159 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/better/business/i-gave-my-credit-card-month-learned-ton-about-my-ncna860801
160 articles downloaded from nbcnews  

209 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/better/lifestyle/many-americans-are-unaware-coronavirus-related-financial-assistance-s-available-ncna1209181
210 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/better/lifestyle/how-not-be-coronavirus-jerk-during-pandemic-ncna1207926
211 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/better/lifestyle/grocery-prices-are-rise-here-are-6-ways-keep-your-ncna1207896
212 articles downloaded from nbcnews  using newspaper, url:  https://www.nbcnews.com/better/lifestyle/cauliflower-tater-tots-4-other-recipes-use-head-cauliflower-ncna1138576
1 articles downloaded from washingtonpost , url:  https://www.washingtonpost.com/world/irans-president-blames-israel-for-killing-nuclear-scientist-and-vows-to-respond-at-the-right-time/2020/11/28/0e0f9576-30f0-11eb-9dd6-2d0179981719_story.html?utm_source=rss&utm_medium=referral&utm_campaign=wp_world
2 article

30  Article has date of type None...
31  Article has date of type None...
32  Article has date of type None...
33  Article has date of type None...
34  Article has date of type None...
35  Article has date of type None...
36  Article has date of type None...
37  Article has date of type None...
38  Article has date of type None...
39  Article has date of type None...
40  Article has date of type None...
41  Article has date of type None...
42  Article has date of type None...
43  Article has date of type None...
44  Article has date of type None...
45  Article has date of type None...
46  Article has date of type None...
47  Article has date of type None...
48  Article has date of type None...
49  Article has date of type None...
50  Article has date of type None...
51  Article has date of type None...
52  Article has date of type None...
53  Article has date of type None...
54  Article has date of type None...
55  Article has date of type None...
Article `download()` failed with HTTPS

In [6]:
with open('scraped_articles.json') as json_data:
    d = json.load(json_data)

In [7]:
for i, site in enumerate((list(d['newspapers']))):
    print(i, site)

0 cnn
1 bbc
2 theguardian
3 breitbart
4 infowars
5 foxnews
6 nbcnews
7 washingtonpost
8 theonion


In [8]:
import pandas as pd
for i, site in enumerate((list(d['newspapers']))):
    articles = list(d['newspapers'][site]['articles'])
    if i == 0:
        df = pd.DataFrame.from_dict(articles)
        df["site"] = site
    else:
        new_df = pd.DataFrame.from_dict(articles)
        new_df["site"] = site
        df = pd.concat([df, new_df], ignore_index = True)     

In [9]:
df.shape

(1454, 5)

In [10]:
df

,title,text,link,published,site
0,Gratitude for Baby Yoda this pandemic year,A version of this story appeared in CNN's Pop ...,http://edition.cnn.com/2020/11/26/entertainmen...,2020-11-26T00:00:00,cnn
1,"Move over, Rudolph: Streaming adds to the floo...",(CNN) Far from the holiday specials of the pas...,http://edition.cnn.com/2020/11/19/entertainmen...,2020-11-19T00:00:00,cnn
2,Courteney Cox recreates Monica's infamous turk...,(CNN) Monica's turkey dance is one of the grea...,http://edition.cnn.com/2020/11/27/entertainmen...,2020-11-27T00:00:00,cnn
3,'The Mandalorian' brings a major fan favorite ...,The following contains spoilers about the late...,http://edition.cnn.com/2020/11/27/entertainmen...,2020-11-27T00:00:00,cnn
4,'The Masked Singer' reveals the legend behind ...,"(CNN) The latest celebrity to be revealed on ""...",http://edition.cnn.com/2020/11/27/entertainmen...,2020-11-27T00:00:00,cnn
...,...,...,...,...,...
1449,Pope elevating 13 new cardinals in ceremony ma...,"The ceremony, known as a consistory, is the se...",https://www.washingtonpost.com/world/the_ameri...,2020-11-28T15:12:21,washingtonpost
1450,Pope elevating 13 new cardinals in ceremony ma...,Cardinals new and old wore facemasks during th...,https://www.washingtonpost.com/world/the_ameri...,2020-11-28T14:53:11,washingtonpost
1451,AP FACT CHECK: Trump distorts military role in...,“That means the full power and strength of the...,https://www.washingtonpost.com/world/national-...,2020-11-28T13:20:25,washingtonpost
1452,Protests over security bill take place across ...,"In Paris, several thousand people packed the s...",https://www.washingtonpost.com/world/europe/pr...,2020-11-28T12:22:21,washingtonpost


In [14]:
import pandas as pd
scraped = pd.DataFrame(df) 
scraped.to_csv(r'C:\Users\Benson\Documents\New FakeNewsDetection\datasets\scraped_data.csv', index=False) 